In [14]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

load_dotenv()

Grog_API_KEY_key = os.getenv("GROQ_API_KEY")


In [15]:
llm = ChatGroq(
    temperature=0.9, 
    groq_api_key=Grog_API_KEY_key, 
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [16]:
from langchain_community.document_loaders import WebBaseLoader

#loader = WebBaseLoader("https://jobs.nike.com/job/R-33460")
loader = WebBaseLoader("https://jobs.careers.microsoft.com/global/en/search?l=en_us&pg=1&pgSz=20&o=Relevance&flt=true&ref=cms")
page_data = loader.load().pop().page_content
print(page_data)


USER_AGENT environment variable not set, consider setting it to identify your requests.


Search Jobs | Microsoft Careers


In [17]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)


str

In [18]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res


[{'role': 'Software Engineer',
  'experience': '2+ years',
  'skills': ['Python', 'Java', 'C++'],
  'description': 'Design, develop, and test software applications'},
 {'role': 'Data Scientist',
  'experience': '5+ years',
  'skills': ['Machine Learning', 'Data Analysis', 'Statistics'],
  'description': 'Analyze and interpret complex data to inform business decisions'},
 {'role': 'Product Manager',
  'experience': '3+ years',
  'skills': ['Product Development', 'Marketing', 'Leadership'],
  'description': 'Lead the development and launch of new products'}]

In [19]:
type(json_res)


list

In [20]:
job = json_res # Assuming you want the first skill from the extracted
job


[{'role': 'Software Engineer',
  'experience': '2+ years',
  'skills': ['Python', 'Java', 'C++'],
  'description': 'Design, develop, and test software applications'},
 {'role': 'Data Scientist',
  'experience': '5+ years',
  'skills': ['Machine Learning', 'Data Analysis', 'Statistics'],
  'description': 'Analyze and interpret complex data to inform business decisions'},
 {'role': 'Product Manager',
  'experience': '3+ years',
  'skills': ['Product Development', 'Marketing', 'Leadership'],
  'description': 'Lead the development and launch of new products'}]

In [22]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df



,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [23]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])


In [24]:
links = collection.query(query_texts=job[0]['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}]]

In [25]:
job[0]

{'role': 'Software Engineer',
 'experience': '2+ years',
 'skills': ['Python', 'Java', 'C++'],
 'description': 'Design, develop, and test software applications'}

In [26]:
job[0]['skills']

['Python', 'Java', 'C++']

In [27]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job[0]), "link_list": links})
print(res.content)


Subject: Expert Software Engineering Services for Your Business Needs

Dear Hiring Manager,

I came across your job description for a Software Engineer with 2+ years of experience in Python, Java, and C++, and I am excited to introduce AtliQ, a leading AI & Software Consulting company that can fulfill your software development needs. Our team of expert software engineers has a proven track record of designing, developing, and testing software applications that drive business growth and optimization.

At AtliQ, we have empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. Our expertise in software development is backed by a strong portfolio of successful projects, including:

* Machine Learning and Python-based solutions: https://example.com/ml-python-portfolio
* Java-based solutions: https://example.com/java-portfolio
* Python-based solutions: https://example.com/python-portfolio

Our team

In [31]:
links = collection.query(query_texts=["Experince in python ", "Expertice in React Native"], n_results=1).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'}]]